In [7]:
import os
import pandas as pd
import yfinance as yf
from ta.momentum import RSIIndicator, stoch
from ta.trend import MACD as md
from ta.volatility import AverageTrueRange as atr
from datetime import datetime

# Technical Scanner

In [8]:
dataframes = {}
buys = {}
sells = {}
buy_list = []
buy_list_sl = []
buy_list_tp = []
sell_list = []
sell_list_tp = []
sell_list_sl = []
buy_list_sq = []
sell_list_sq = []
buy_list_osq = []
sell_list_osq = []
on_squeeze = []
outof_squeeze = []

with open('ticker.csv') as f:
    lines = f.read().splitlines()
    for symbol in lines:
        print(symbol)
        data = yf.download(symbol, start="2020-01-01", end=datetime.today().strftime('%Y-%m-%d'))
        df = pd.DataFrame(data)

        RSI = RSIIndicator(close=df['Close'], n=7)
        df["RSI"] = RSI.rsi()

        Stochk = stoch(high=df["High"],low=df["Low"],close=df["Close"]).rolling(window=3).mean()
        df["Stochk"] = Stochk

        MACD = md(close=df['Close'])
        df["MACD_diff"] = MACD.macd_diff()

        AvgTru = atr(high=df['High'],low=df['Low'],close=df['Close'],n=7)
        df["ATR"] = AvgTru.average_true_range()

        df["Signal"] = ""

        df['20sma'] = df['Close'].rolling(window=20).mean()
        df['stddev'] = df['Close'].rolling(window=20).std()
        df['lower_band'] = df['20sma'] - (2 * df['stddev'])
        df['upper_band'] = df['20sma'] + (2 * df['stddev'])

        df['TR'] = abs(df['High'] - df['Low'])
        df['ATR'] = df['TR'].rolling(window=20).mean()

        df['lower_keltner'] = df['20sma'] - (df['ATR'] * 1.5)
        df['upper_keltner'] = df['20sma'] + (df['ATR'] * 1.5)

        def in_squeeze(df):
            return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']

        df['squeeze_on'] = df.apply(in_squeeze, axis=1)

        if df["squeeze_on"].iloc[-1]==True:
            on_squeeze.append(symbol)

        if (df["squeeze_on"].iloc[-1]==True) and (df["squeeze_on"].iloc[-2]==False):
            outof_squeeze.append(symbol)

        for i in range(0,len(df)):
            if df["RSI"].iloc[i] > 50 and df["Stochk"][i] > 50 and df["MACD_diff"][i] > 0:
                df["Signal"][i] = "Buy"
            elif df["RSI"].iloc[i] < 50 and df["Stochk"][i] < 50 and df["MACD_diff"][i] < 0:
                df["Signal"][i] = "Sell"
            else:
                df["Signal"][i] = "Neutral"

            if df["Signal"].iloc[-1] == "Buy" and (df["Signal"].iloc[-2] != "Buy"):
                buy_list.append(symbol)
                buy_list_sl.append(df["Close"].iloc[-1]-(df["ATR"].iloc[-1]*1.3))
                buy_list_tp.append(df["Close"].iloc[-1]+(df["ATR"].iloc[-1]*2.2))
                buy_list_sq.append(df['squeeze_on'].iloc[-1])
            elif df["Signal"].iloc[-1] == "Sell" and (df["Signal"].iloc[-2] != "Sell"):
                sell_list.append(symbol)
                sell_list_sl.append(df["Close"].iloc[-1]+(df["ATR"].iloc[-1]*1.3))
                sell_list_tp.append(df["Close"].iloc[-1]-(df["ATR"].iloc[-1]*2.2))
                sell_list_sq.append(df['squeeze_on'].iloc[-1])
            else:
                continue

        buys = pd.DataFrame({
            "Symbol" : buy_list,
            "SL" : buy_list_sl,
            "TP" : buy_list_tp,
            "Squeeze" : buy_list_sq
        })

        sells = pd.DataFrame({
            "Symbol" : sell_list,
            "SL" : sell_list_sl,
            "TP" : sell_list_tp,
            "Squeeze" : sell_list_sq
        })

        oos = pd.DataFrame({
            "Out of squeeze" : outof_squeeze
        })

buys.to_csv("buys.csv")
sell.to_csv("sells.csv")
oos.to_csv("oos.csv")

BIG
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed


KeyError: 'outof_squeeze'

In [9]:
on_squeeze

['PFSI']

In [3]:
oos

,Out of squeeze


# Options Delta Identifier

In [5]:
import tdameritrade as td

client_id = "DRSMU4TL964FO3QNBQHVL78X9SUPGGIL"
account_id = None
refresh_token = "9WBdSYb/o0eDYM6g9bQbdCOGFhWcg0ZloP5KfWdBWzeRNfRTqjnl6M9zzh4zr1oXzxk1EKIqzip77j7NqMcn160ZoHVBPOWmLMxiSpBaXVnBHcuk5VGBnNiPpFxbP5Ex1/nQx2nM3YvM/byNDpkI0zbRFvhsNMOClslJ+nyj2VGC167FHDTXIP3cLl1nQyd39WdWnafgXOIcVjZCOqGj+cIguNKv2NImj4OJSFF9BO2npMrbRyESGhHmTxPb3EMPPhdq18QDspBUfpeyVuO4Hwl7JnhbhUnZWRYSt0Gu0Phx6anlLkY641T54FngRvdHC9pobhSPTmSB9KOEL9IZjOTgNr0llWxzaTliuQdPsXYJizsjc42TkO9fc+YvGAyKm+oFYLbAqo1XZzODzi3lh2TW85eDbc+6MriMtmqSnMaRmB39cxwUnh5CGXj100MQuG4LYrgoVi/JHHvl8nshsQUyVECXgHrbWCaczm6Axpp0uQmOhwzJiv7v22TcHw7GbIH51GZjuImaWzcgQu5GTofWx04eTCBQxsT0Hvs0QPMmJiur59bBNntVYp25Ns0iQ2myTAscfaTGMqZbXJNI9Rcb2E2gg1UFMJQzbtQtjc8bmY66Udj59VF7Jt8XJDJdeggesk7vOzDFdKG46vl14Liw8AbGpboh4KWl444OWPtxPF8EMfamMESsYsnjzlP7BXn7j398gHBXhmG9a3M6PS7yw/FvE9gjFI+NPGf9USruBtcvYEguywHh2i2Y0/FyDgG9f3Yt+LWQjiv90w1gnEjP6yPARboFaXuw0y42DMvGbXuCYAZKCzkWgIuKJObiIKBNo9uASJ1nWt9B0KPDJrHAG352B4zJ3q6+bjjcQjYZ4+ann1bL0VcVU0bexhGyAoeDuyYmgos=212FD3x19z9sWBHDJACbC00B75E"

c = td.TDClient(client_id=client_id, refresh_token=refresh_token)
delta_est = 0.70

deltal = []
voll = []
pricel = []
strikel = []

for stk in on_squeeze:
    df = c.optionsDF(stk,contractType='CALL', strikeCount=25, includeQuotes=True, strategy='SINGLE', interval=None, strike=None, range='ALL', fromDate="2020-12-17", toDate="2020-12-19", volatility=None, underlyingPrice=None, interestRate=None, daysToExpiration=None, expMonth='ALL', optionType='ALL')

    narrow_df =  df[["strikePrice","delta","volatility","bid","ask","closePrice"]]
    idx = narrow_df['delta'].sub(delta_est).abs().idxmin()
    filtered_df = narrow_df.loc[[idx]]
    deltal.append(float(filtered_df["delta"]))
    voll.append(float(filtered_df["volatility"]))
    pricel.append(float(filtered_df["closePrice"]))
    strikel.append(float(filtered_df["strikePrice"]))

In [8]:
squeezeOpt = pd.DataFrame({"Ticker" : on_squeeze, "Strike" : strikel, "Delta" : deltal, "Price" : pricel, "IV" : voll})
squeezeOpt

,Ticker,Strike,Delta,Price,IV
0,BIG,47.5,0.707,6.46,76.761
1,DG,207.5,0.723,12.68,32.017
2,XP,40.0,0.678,3.05,46.174
3,ATVI,73.5,0.710,4.15,30.656
4,PETS,30.0,0.547,1.42,43.174


# Options Profit Calculator

In [44]:
import mibian as mb
import requests
from bs4 import BeautifulSoup

TREASURY_URL = "https://www.marketwatch.com/investing/bond/tmubmusd03m?countrycode=bx"
OVERNIGHT_RATE = 0

r = requests.get(TREASURY_URL)
soup = BeautifulSoup(r.text, 'html.parser')

rfr = float(soup.find('bg-quote',{"class": "value"}).string)

In [57]:
sim_list = []
underlying_price = []

for i in range(355,400,2):
    underlying_price.append(i)
    sim = mb.BS([i,float(narrow_df["strikePrice"]),rfr,2],volatility=float(narrow_df["volatility"]))
    returns = (sim.callPrice - float(narrow_df["closePrice"]))*100
    sim_list.append(returns)
    track = pd.DataFrame(
            {"Underlying" : underlying_price,
            "Returns" : sim_list
            }
    )

In [58]:
import plotly.express as px

In [2]:
import tdameritrade as td

In [59]:
px.line(track, x="Underlying", y="Returns")